In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import mnist

In [2]:
class Mnist(torch.utils.data.Dataset):
    def __init__(self, limit, train = True):
        if train:
            data, labels, _, _ = mnist.load()
        else:
            _, _, data, labels = mnist.load()
        print(data.shape)
        self.data = data
        self.labels = labels
        self.limit = limit
    
    def __len__(self):
        return len(self.data[:self.limit])

    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx].reshape(1,28,28)).float()/255, self.labels[idx]  

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnv = nn.Conv2d(1, 32, (5, 5))
        self.pool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(32*12**2, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.cnv(x))
        x = self.flat(self.pool(x))
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
model = Net() # Формируем модель НСpass
optim = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss(reduction="sum")

trainloader = torch.utils.data.DataLoader(Mnist(10000), batch_size=1, shuffle = False)
testloader = torch.utils.data.DataLoader(Mnist(1000, False), batch_size=1)

(60000, 784)
(10000, 784)


In [4]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optim.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optim.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 1.331
[1,  4000] loss: 0.457
[1,  6000] loss: 0.377
[1,  8000] loss: 0.335
[1, 10000] loss: 0.335
[2,  2000] loss: 0.294
[2,  4000] loss: 0.195
[2,  6000] loss: 0.199
[2,  8000] loss: 0.197
[2, 10000] loss: 0.198
[3,  2000] loss: 0.168
[3,  4000] loss: 0.112
[3,  6000] loss: 0.120
[3,  8000] loss: 0.120
[3, 10000] loss: 0.125
Finished Training


In [5]:
results = np.zeros((3, 10))
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients

        # forward + backward + optimize
        outputs = model(inputs)
        results[0][labels[0]] += 1
        pr = torch.argmax(outputs, dim=1)
        if(pr == labels):
            results[1][labels[0]] +=1
        results[2][pr[0]] += 1
print(results[0])
print(results[1])
print(results[2])
print(results[1]/results[0])

[ 85. 126. 116. 107. 110.  87.  87.  99.  89.  94.]
[ 84. 124. 106.  99. 104.  82.  84.  92.  87.  88.]
[ 88. 126. 110. 101. 106.  85.  89.  97. 103.  95.]
[0.98823529 0.98412698 0.9137931  0.92523364 0.94545455 0.94252874
 0.96551724 0.92929293 0.97752809 0.93617021]


In [6]:
np.sum(results[1])/np.sum(results[0])

0.95